In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
from report_utils import excel_report

import numpy as np
from log import log

In [2]:
sql = '''
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case 
        	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
        	else 'ПРОЧИЕ' 
       end "name",
	   DATE(a.finished + 3 * interval '1 hour')
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join promotion promo on a.promotion_id = promo.id
where a.finished at time zone 'UTC' > '2020-03-30'
and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second')
and a.good
	and not (
			a.patient_id in (
							select patient_id from patient_categories pcat
							inner join reference rf 
							on rf.id = pcat.reference_id
							where rf.code = 'TEST'
			)
			or lower(a.report_comment) = 'тест'
	)

'''

In [3]:
sql = '''
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case 
        	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
        	else 'ПРОЧИЕ' 
       end "name",
	   DATE(a.finished + 3 * interval '1 hour')
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join promotion promo on a.promotion_id = promo.id
where a.finished at time zone 'UTC' > '2020-03-30'
and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second' - 9 * interval '1 hour')
and a.good
	and not (
			a.patient_id in (
							select patient_id from patient_categories pcat
							inner join reference rf 
							on rf.id = pcat.reference_id
							where rf.code = 'TEST'
			)
			or lower(a.report_comment) = 'тест'
	)

'''

In [4]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-04-30 11:19:35 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-04-30 11:19:35 creating ssh tunnel to 172.16.100.19 as root...
2020-04-30 11:19:35 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 44835
2020-04-30 11:19:35 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [5]:
data = connection.query(sql)

2020-04-30 11:19:35 @telemed query:
                    select a.id appoint_id,
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case 
				            	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
				            	else 'ПРОЧИЕ' 
				           end "name",
				    	   DATE(a.finished + 3 * interval '1 hour')
				    from appointment a
				    left join patient p on a.patient_id = p.id 
				    left join person per on p.person_id = per.id 
				    left join promotion promo on a.promotion_id = promo.id
				    where a.finished at time zone 'UTC' > '2020-03-30'
				    and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second' - 9 * interval '1 hour')
				    and a.good
				    	and not (
				    			a.patient_id in (
				    							select patient_id from patient_catego

In [6]:
connection.close()

2020-04-30 11:19:37 @telemed: closing postgres connection...
2020-04-30 11:19:37 closing ssh tunnel to 172.16.100.19...


In [7]:
df = pd.DataFrame(data)

In [8]:
df.head()

,appoint_id,person_id,phone,name,date
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08
1,127919,2546091,79518565014,VEBMED,2020-03-31
2,144566,3521114,79687101977,VEBMED,2020-04-15
3,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07
4,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31


----------

In [9]:
def phone_base():
    base = pd.read_csv('DEF.csv')
    
    column_base = ['code', 'start', 'end', 'capacity', 'operator', 'region']
    base.columns = column_base
    
    return base

In [10]:
def get_region_by_phone(phone, base=phone_base()):
    if type(phone) != str:
        phone = str(phone)
    phone = ''.join(i for i in phone if i.isdigit())    
    if phone[0] == '8' or phone[0] == '7':
        phone = phone[1:]
    if len(phone) != 10:
        return 'Неизвестный регион'
    return base.loc[(base['code'] == int(phone[0:3])) & 
                          (base['start'] <= int(phone[3:])) &
                          (base['end'] >= int(phone[3:]))
                         ]['region'].to_string(index=False).strip()

In [11]:
#get_region_by_phone('9011009999')

In [12]:
#get_region_by_phone('+79260001133')

In [13]:
#get_region_by_phone('7(928)997-00-00')

In [14]:
#get_region_by_phone(89011009999)

In [15]:
#df_test = df.sort_values(by='date').reset_index()

In [16]:
#df_test

In [17]:
#df_test.loc[0]['date']

In [18]:
#df_test[(df_test['date'] == df_test.loc[0]['date']) & (df_test['region'] == 'Алтайский край')]

In [19]:
df['region'] = df['phone'].apply(get_region_by_phone)

In [20]:
df = df.replace('Series([], )', 'Неизвестный регион')

In [21]:
df.head(15)

,appoint_id,person_id,phone,name,date,region
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08,г. Москва и Московская область
1,127919,2546091,79518565014,VEBMED,2020-03-31,Воронежская обл.
2,144566,3521114,79687101977,VEBMED,2020-04-15,г. Москва и Московская область
3,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07,Свердловская обл.
4,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31,Краснодарский край
5,135952,3201364,79911116364,VEBMED,2020-04-07,Ставропольский край
6,168719,3565800,79520769150,ПРОЧИЕ,2020-04-28,Республика Мордовия
7,152628,3531222,79536239692,VEBMED,2020-04-21,Орловская обл.
8,127960,2626621,79149123969,ПРОЧИЕ,2020-03-31,Иркутская обл.
9,127116,2632032,79002066203,ПРОЧИЕ,2020-03-31,Свердловская обл.


In [22]:
#df[(df['date'] == '2020-03-30') & (df['region'] == 'Алтайский край')]

In [23]:
#df.info()

In [24]:
reg_names = [name for name in df.region.unique()]

In [25]:
reg_names.sort()

In [26]:
reg_names.insert(0, 'Всего консультаций')

In [27]:
#reg_names

In [28]:
#reg_names.index('Incorrect phone')

In [29]:
#d = dict()

In [30]:
#for i in range(df.shape[0]):
#    d[df['date'][i]] = d.get(df['date'][i], [0 for name in reg_names]) 
#    d[df['date'][i]][reg_names.index(df['region'][i])] += 1
#    d[df['date'][i]][0] += 1

In [31]:
#d

In [32]:
#report = pd.DataFrame.from_dict(d, orient='index').sort_index()

In [33]:
#report.columns = reg_names

In [34]:
#report

In [35]:
d2 = dict()

In [36]:
dates = [date for date in df.date.unique()]

In [37]:
dates.sort()

In [38]:
dates

[datetime.date(2020, 3, 30),
 datetime.date(2020, 3, 31),
 datetime.date(2020, 4, 1),
 datetime.date(2020, 4, 2),
 datetime.date(2020, 4, 3),
 datetime.date(2020, 4, 4),
 datetime.date(2020, 4, 5),
 datetime.date(2020, 4, 6),
 datetime.date(2020, 4, 7),
 datetime.date(2020, 4, 8),
 datetime.date(2020, 4, 9),
 datetime.date(2020, 4, 10),
 datetime.date(2020, 4, 11),
 datetime.date(2020, 4, 12),
 datetime.date(2020, 4, 13),
 datetime.date(2020, 4, 14),
 datetime.date(2020, 4, 15),
 datetime.date(2020, 4, 16),
 datetime.date(2020, 4, 17),
 datetime.date(2020, 4, 18),
 datetime.date(2020, 4, 19),
 datetime.date(2020, 4, 20),
 datetime.date(2020, 4, 21),
 datetime.date(2020, 4, 22),
 datetime.date(2020, 4, 23),
 datetime.date(2020, 4, 24),
 datetime.date(2020, 4, 25),
 datetime.date(2020, 4, 26),
 datetime.date(2020, 4, 27),
 datetime.date(2020, 4, 28),
 datetime.date(2020, 4, 29)]

In [39]:
for i in range(df.shape[0]):
    d2[df['region'][i]] = d2.get(df['region'][i], [0 for date in dates]) 
    d2[df['region'][i]][dates.index(df['date'][i])] += 1
    #d2[df['region'][i]][0] += 1

In [40]:
#d2

In [41]:
report2 = pd.DataFrame.from_dict(d2, orient='index').sort_index()

In [42]:
report2.columns = dates

In [43]:
report2['mean'] = report2.mean(axis=1).astype(int)

In [44]:
report2['total'] = report2.sum(axis=1)

In [45]:
report2.sort_values(report2.columns[10], ascending=False).head(25)
#report2.sort_values('mean', ascending=False).head(25)

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,mean,total
г. Москва и Московская область,172,201,241,250,285,226,212,288,263,308,...,509,519,544,461,394,583,565,327,353,11321
Орловская обл.,6,3,1,1,5,3,1,3,2,20,...,14,12,14,16,15,24,30,20,17,565
г. Санкт-Петербург и Ленинградская область,52,60,81,83,76,64,69,79,65,75,...,114,106,128,94,94,118,118,57,84,2716
Ивановская обл.,6,6,7,6,7,7,6,11,55,139,...,55,57,113,62,44,53,66,25,45,1443
Республика Татарстан,10,19,14,13,24,19,64,58,42,49,...,65,83,95,87,71,94,107,56,45,1443
Ярославская обл.,2,3,11,19,7,8,14,20,26,23,...,19,27,18,19,15,34,27,17,21,684
Свердловская обл.,12,16,21,28,25,27,19,25,23,24,...,39,42,43,29,24,36,48,26,26,853
Нижегородская обл.,16,16,15,18,14,18,12,22,15,13,...,44,112,96,42,41,70,68,26,29,946
Республика Башкортостан,15,9,21,13,12,34,29,32,38,29,...,68,52,50,31,37,38,46,17,30,979
Краснодарский край,16,23,23,18,23,17,18,35,16,27,...,38,52,56,53,53,67,51,21,29,931


-------

In [46]:
#report3.loc[['Ивановская обл. VEBMED', 'Ивановская обл. _прочие']]

In [47]:
#report3.loc[['Орловская обл. VEBMED', 'Орловская обл. _прочие']]

In [48]:
#report3.loc[['Республика Татарстан VEBMED', 'Республика Татарстан _прочие']]

In [49]:
report2 = report2.sort_values('total', ascending=False)

In [50]:
#report2.to_excel('app_regions_new.xls')

----

# WEBMED

In [51]:
d3 = dict()

In [52]:
dates = [date for date in df.date.unique()]
dates.sort()

In [53]:
#df['region'][1]

In [54]:
for i in range(df.shape[0]):
    if df['name'][i] == 'VEBMED':
        #region_name = df[]
        d3[df['region'][i] + ' VEBMED'] = d3.get(df['region'][i]  + ' VEBMED', [0 for date in dates]) 
        d3[df['region'][i] + ' VEBMED'][dates.index(df['date'][i])] += 1
    #d2[df['region'][i]][0] += 1
    
    if df['name'][i] != 'VEBMED':
        d3[df['region'][i] + ' _прочие'] = d3.get(df['region'][i] + ' _прочие', [0 for date in dates]) 
        d3[df['region'][i] + ' _прочие'][dates.index(df['date'][i])] += 1

In [55]:
report3 = pd.DataFrame.from_dict(d3, orient='index').sort_index()

In [56]:
report3.columns = dates

In [57]:
report3['total'] = report3.sum(axis=1)

In [58]:
report3

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,total
Алтайский край VEBMED,0,1,2,1,6,1,2,5,3,2,...,8,4,25,27,31,34,32,18,10,266
Алтайский край _прочие,4,1,1,2,5,3,3,7,3,6,...,2,2,2,1,1,1,2,0,0,84
Амурская обл. VEBMED,0,0,1,1,2,1,0,0,0,2,...,3,2,2,1,2,3,3,3,2,40
Амурская обл. _прочие,2,0,0,0,0,0,0,3,2,0,...,0,0,0,1,0,1,0,5,0,23
Архангельская обл. VEBMED,0,0,1,0,0,1,0,2,1,0,...,4,1,5,1,1,0,0,1,1,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
г. Москва и Московская область _прочие,157,150,173,178,207,144,112,155,152,184,...,254,229,234,228,186,152,246,216,138,5840
г. Санкт-Петербург и Ленинградская область VEBMED,1,10,18,15,16,26,22,30,27,18,...,67,62,50,76,57,64,69,69,34,1070
г. Санкт-Петербург и Ленинградская область _прочие,51,50,63,68,60,38,47,49,38,57,...,48,52,56,52,37,30,49,49,23,1562
г. Сочи|Краснодарский край VEBMED,0,0,0,0,1,0,0,0,0,0,...,0,0,3,1,2,3,4,4,3,25


In [59]:
#report3.to_excel('app_regions_new-3.xls')

In [60]:
file_name = 'appointments_regions_30-04.xls'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
report3.to_excel(writer, sheet_name='WEB partition')
report2.to_excel(writer, sheet_name='Regions total')

writer.save()